In [ ]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
from collections.abc import MutableMapping
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar
import time
import re

import nltk
#nltk.download()
#nltk.download('stopwords')
from nltk.corpus import stopwords
import advertools as adv

#Logger
logging.basicConfig(filename='Text-BioAnalysis.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
##############
# PARAMETERS #
##############

client = MongoClient(username='XXX', password='XXX')

DatabaseName = "Streaming"
CollectionName = "Users"

db = client[DatabaseName]

In [ ]:
#################
# GET BIO TEXT  #
#################

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
                           "]+", flags=re.UNICODE)

def clean_tweet(twit_text):
    twit_text_filtered = re.sub(r'[#@]', '', twit_text)
    twit_text_filtered_2 = re.sub(r'(\s)http\S+', '', twit_text_filtered)
    twit_text_filtered_3 = emoji_pattern.sub(r'', twit_text_filtered_2)
    #twit_text_filtered_3 = twit_text_filtered_2.encode('ascii', 'ignore').decode('ascii') 
    return twit_text_filtered_3


def load_users(user_collection):

    pipeline = [
                {
                    '$match': {
                        'description': {            
                            '$exists': True
                        },
                    }
                }, {
                    '$project': {
                        '_id': True, 
                        'description': True
                    }
                }
            ]
    print("Query", end=" ")
    users = user_collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    users = list(users)
    print("OK; Total users:", len(users))
    return users


users_list = load_users (db[CollectionName])

text_total = ""
for user in progress_bar(users_list):
    text_total += " " + clean_tweet(str(user['description']))
            
print(len(text_total))

In [ ]:
print(len(text_total.split()))

tokens = [w.strip('“”.,;:-():!?-‘’|/•&+* ') for w in text_total.lower().split()]

# Create counter
counts = Counter(tokens)
print(len(counts))
counts.most_common(150)

In [ ]:
#####################
# REMOVE STOPWORDS  #
#####################

stopwords_spanish = nltk.corpus.stopwords.words('spanish')
stopwords_english = nltk.corpus.stopwords.words('english')
stopwords_catalan = adv.stopwords['catalan']

custom_stopwords = ['none', '', 'q', 'l', '\n']

stopwords = stopwords_spanish + stopwords_english + list(stopwords_catalan)
stopwords.extend(custom_stopwords)

important_tokens = [important_token for important_token in tokens if important_token not in stopwords]
print(len(important_tokens))

# Create counter
counts = Counter(important_tokens)
print(len(counts))
counts.most_common(150)

In [ ]:
####################################################
# BIO ANALYSIS BY COMMUNITY IMPROVED (apostrophe)  #
####################################################

# MADRID
COMMUNITIES_LIST = ["VOX", "PODEMOS", "PSOE", "PP", "MAS_MAD", "CS"]

# ANDALUSIA
# COMMUNITIES_LIST = ["VOX", "POR_AND", "PSOE", "PP", "ADELANTE_AND", "CS"]

def load_users_by_community(user_collection, community):
    """Extracts the ObjectID and created_at of users
    
    Keyword arguments:
    user_collection -- MongoDB Users' Collection
    """
    pipeline = [
                {
                    '$match': {
                        'community': community,
                    }
                }, {
                    '$project': {
                        '_id': True, 
                        'description': True
                    }
                }
            ]
    print("Query", end=" ")
    users = user_collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    users = list(users)
    print("OK; Total users:", len(users))
    return users

for community in COMMUNITIES_LIST:
    print("******************************************************")
    print("                      " + community)
    print("******************************************************")
    users_by_community_list = load_users_by_community(db[CollectionName], community)

    text_total = ""
    for user in users_by_community_list:
        text_total += " " + clean_tweet(str(user['description']))

    tokens = [w.strip('“”.,;:-():!?-‘’|/•&+* ') for w in re.split(r"[ ']+", text_total.lower())]   
    important_tokens = [important_token for important_token in tokens if important_token not in stopwords]

    counts = Counter(important_tokens)
    print("Total Tokens: " + str(len(counts)))
    print(counts.most_common(50))